<a href="https://colab.research.google.com/github/blue0620/simple_reading_order/blob/main/NDLOCR_googlecolabversion_reading_order.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 概要

現行のNDLOCR(https://github.com/ndl-lab/ndlocr_cli)
には読み順でソートする機能がないことについて不便であろうと思いますので、読み順付与機能を作ってみました。

OCRデータセットのレイアウト座標データに対して、ランク学習(LambdaRank)を適用することで、レイアウト間の配置と読みの順序の対応を学習させています。

ランク学習については
https://www.szdrblog.info/entry/2018/12/05/001732
等を参考にしていただければと思います。(このアドベントカレンダー自体が神資料です。@sz_drさん、ありがとうございます！)

これにより、未知のレイアウトに対しても、妥当な読み順を自動付与してくれる機能を追加しています。

## この追加機能の精度
スピアマンの順位相関係数で測定すると**0.98程度**なので、相当正確ではないかと思います。多段組の資料でも処理可能ですが、割書きや図表キャプションが多く含まれる資料は苦手のようです。

## 処理時間
最大のアピールポイントで、CPUのみ利用で1紙面あたり **30ミリ秒 (0.03秒)程度** ですので、これまでのNDLOCRの利用体験を損なわずに利用可能です。

この秘密はOCRが出力した座標情報から作成した次の13特徴量から学習したブースティング木(lightGBM)をモデルに採用しているためです。


「左上正規化x座標」,「左上正規化y座標」,「正規化レイアウト幅」,「正規化レイアウト高さ」,「左側に隣接する他レイアウトとの正規化距離(なければ2)」,「右側に隣接する他レイアウトとの正規化距離(なければ2)」,「上側に隣接する他レイアウトとの正規化距離(なければ2)」,「下側に隣接する他レイアウトとの正規化距離(なければ2)」,「自分からみて左側に存在するレイアウトの個数」,「自分からみて右側に存在するレイアウトの個数」,「自分からみて上側に存在するレイアウトの個数」,「自分からみて下側に存在するレイアウトの個数」,「縦書きか横書きか」


## 余談
単に各レイアウトの正規化x座標、y座標を特徴量にして同じアルゴリズムを適用しても大した精度は出ず、他のレイアウトとの関係性を表す特徴量が重要です。なぜなのか考察すると面白いと思います。


タスクの性質上、今回のLambdaRankのようなpairwiseなランク学習手法よりもListNetのようなlistwiseなランク学習手法の方がパフォーマンス出そうな気もしますが、面倒になったのでやりません。どなたか拡張してください！！(他力本願)

Written by Toru Aoike(@blue0620)

# 0. GPUの情報を確認する

In [2]:
!nvidia-smi

Mon May  2 13:58:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 1. NDLOCRのリポジトリをcloneする(--recursiveを忘れずに！)

In [3]:
!git clone --recursive https://github.com/ndl-lab/ndlocr_cli

Cloning into 'ndlocr_cli'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 114 (delta 61), reused 33 (delta 9), pack-reused 0
Receiving objects: 100% (114/114), 66.60 KiB | 5.12 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Submodule 'src/deskew_HT' (https://github.com/ndl-lab/deskew_HT.git) registered for path 'src/deskew_HT'
Submodule 'src/ndl_layout' (https://github.com/ndl-lab/ndl_layout.git) registered for path 'src/ndl_layout'
Submodule 'src/separate_pages_ssd' (https://github.com/ndl-lab/separate_pages_ssd.git) registered for path 'src/separate_pages_ssd'
Submodule 'src/text_recognition' (https://github.com/ndl-lab/text_recognition.git) registered for path 'src/text_recognition'
Cloning into '/content/ndlocr_cli/src/deskew_HT'...
remote: Enumerating objects: 35, done.        
remote: Counting objects: 100% (35/35), done.        
remote: Compressing objects: 100% (35/35

# 2. 必要なパッケージをインストールする

In [ ]:
#!cp /content/drive/MyDrive/inference.py /content/ndlocr_cli/cli/core/
#!cp /content/ndlocr_cli/cli/core/inference.py  /content/drive/MyDrive/

In [4]:
PROJECT_DIR="/content/ndlocr_cli"

In [5]:
!pip install -r {PROJECT_DIR}/requirements.txt
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip install mmcv-full==1.4.0 -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
##numpyのバージョン問題でcolabでは動かなかったのでアップデートする(参考:https://stackoverflow.com/questions/66060487/valueerror-numpy-ndarray-size-changed-may-indicate-binary-incompatibility-exp)
#!pip install --upgrade numpy

     |████████████████████████████████| 312 kB 14.9 MB/s 
     |████████████████████████████████| 299 kB 67.7 MB/s 
     |████████████████████████████████| 1.5 MB 64.5 MB/s 
     |████████████████████████████████| 14.8 MB 56.1 MB/s 
     |████████████████████████████████| 50.4 MB 40 kB/s 
     |████████████████████████████████| 26.5 MB 99.6 MB/s 
     |████████████████████████████████| 31.2 MB 1.4 MB/s 
     |████████████████████████████████| 3.1 MB 61.9 MB/s 
     |████████████████████████████████| 109.3 MB 61 kB/s 
     |████████████████████████████████| 488 kB 66.0 MB/s 
     |████████████████████████████████| 50 kB 9.2 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  At

Looking in links: https://download.pytorch.org/whl/lts/1.8/torch_lts.html
     |█████████████▌                  | 834.1 MB 1.3 MB/s eta 0:14:46tcmalloc: large alloc 1147494400 bytes == 0x2fb8000 @  0x7fc176516615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.4 MB/s eta 0:11:23tcmalloc: large alloc 1434370048 bytes == 0x4760e000 @  0x7fc176516615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████████▋          | 1336.2 MB 1.2 MB/s eta 0:09:12tcmalloc: large alloc 1792966656 byt

In [6]:
%cd {PROJECT_DIR}/src/ndl_layout/mmdetection
!python setup.py bdist_wheel
!pip install dist/*.whl
%cd /content

/content/ndlocr_cli/src/ndl_layout/mmdetection
running bdist_wheel
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:369: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
running build
running build_py
creating build
creating build/lib
creating build/lib/mmdet
copying mmdet/__init__.py -> build/lib/mmdet
copying mmdet/version.py -> build/lib/mmdet
creating build/lib/mmdet/apis
copying mmdet/apis/test.py -> build/lib/mmdet/apis
copying mmdet/apis/__init__.py -> build/lib/mmdet/apis
copying mmdet/apis/train.py -> build/lib/mmdet/apis
copying mmdet/apis/inference.py -> build/lib/mmdet/apis
creating build/lib/mmdet/utils
copying mmdet/utils/logger.py -> build/lib/mmdet/utils
copying mmdet/utils/profiling.py -> build/lib/mmdet/utils
copying mmdet/utils/__init__.py -> build/lib/mmdet/utils
copying mmdet/utils/util_mixins.py

# 4. OCRに必要な学習済みモデルをダウンロードする

In [7]:
%cd {PROJECT_DIR}
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth -P ./src/text_recognition/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/ndl_layout_config.py -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/ndl_layout/epoch_140_all_eql_bt.pth -P ./src/ndl_layout/models
!wget https://lab.ndl.go.jp/dataset/ndlocr/separate_pages_ssd/weights.hdf5 -P ./src/separate_pages_ssd/ssd_tools
%cd /content/

/content/ndlocr_cli
--2022-05-02 14:04:11--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/mojilist_NDL.txt
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 108.157.4.56, 108.157.4.37, 108.157.4.84, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|108.157.4.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28088 (27K) [text/plain]
Saving to: ‘./src/text_recognition/models/mojilist_NDL.txt’

mojilist_NDL.txt    100%[===================>]  27.43K   122KB/s    in 0.2s    

2022-05-02 14:04:13 (122 KB/s) - ‘./src/text_recognition/models/mojilist_NDL.txt’ saved [28088/28088]

--2022-05-02 14:04:13--  https://lab.ndl.go.jp/dataset/ndlocr/text_recognition/ndlenfixed64-mj0-synth1.pth
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 108.157.4.56, 108.157.4.37, 108.157.4.84, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|108.157.4.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191768312 (183M) [application/x-www-form-urlencoded]
Saving to: ‘./

# 4.5. 拡張機能【読み順の自動ソート】を追加する(NEW!)

後処理の追加のため、
/content/ndlocr_cli/cli/core/inference.py

を置き換える必要があります。


In [9]:
%cd {PROJECT_DIR}
!rm ./cli/core/inference.py
!wget https://raw.githubusercontent.com/blue0620/simple_reading_order/main/inference.py -P ./cli/core/
!wget https://lab.ndl.go.jp/dataset/ndlocr/appendix/simple_reading_order_model.joblib -P .
%cd /content/

/content/ndlocr_cli
rm: cannot remove './cli/core/inference.py': No such file or directory
--2022-05-02 14:05:39--  https://raw.githubusercontent.com/blue0620/simple_reading_order/main/inference.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25812 (25K) [text/plain]
Saving to: ‘./cli/core/inference.py’

inference.py        100%[===================>]  25.21K  --.-KB/s    in 0s      

2022-05-02 14:05:39 (117 MB/s) - ‘./cli/core/inference.py’ saved [25812/25812]

--2022-05-02 14:05:39--  https://lab.ndl.go.jp/dataset/ndlocr/appendix/simple_reading_order_model.joblib
Resolving lab.ndl.go.jp (lab.ndl.go.jp)... 54.192.87.15, 54.192.87.32, 54.192.87.4, ...
Connecting to lab.ndl.go.jp (lab.ndl.go.jp)|54.192.87.15|:443... connected.
HTTP request sent, awaiting r

# 5. 環境変数を追加する

In [10]:
import os
os.environ["PYTHONPATH"]=os.environ["PYTHONPATH"]+":"+f"{PROJECT_DIR}/src/text_recognition/deep-text-recognition-benchmark"

# 6. PDFを画像に変換するためのパッケージのインストール

In [11]:
!apt-get install poppler-utils
!pip install pdf2image 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.12 [154 kB]
Fetched 154 kB in 0s (1,835 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 155202 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.12_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.12) ...
Setting up poppler-utils (0.62.0-2ubuntu2.12) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# 7. テキスト化したいPDFをダウンロードする
今回は、ROIS-DS人文学オープンデータ共同利用センター(http://codh.rois.ac.jp/
)が提供している

近代雑誌データセット　http://codh.rois.ac.jp/modern-magazine/
から、

東洋学芸雑誌(https://dglb01.ninjal.ac.jp/ninjaldl/bunken.php?title=toyogakuge)

第一号(https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf)

をダウンロードしてみます。

In [12]:
!curl https://dglb01.ninjal.ac.jp/ninjaldl/toyogakuge/001/PDF/tygz-001.pdf -o /content/tygz-001.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.1M  100 14.1M    0     0  4177k      0  0:00:03  0:00:03 --:--:-- 4177k


# 8. PDFをjpeg画像に変換する

In [13]:
from pathlib import Path
from pdf2image import convert_from_path
import os
pdf_path = Path("/content/tygz-001.pdf")
os.makedirs("/content/tygz-001/img",exist_ok=True)
img_path=Path("/content/tygz-001/img")

convert_from_path(pdf_path, output_folder=img_path,fmt='jpeg',output_file=pdf_path.stem,dpi=100)

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF4888F410>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47160B90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169850>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169950>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169A50>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169B90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169C90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169D90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169E90>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47169B50>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3705x5173 at 0x7EFF47172090>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB siz

# 9. OCRの実行

/content/tygz-001以下のimgディレクトリ内の画像を処理し、
/content/tygz-001_outputに出力する場合

In [14]:
%cd {PROJECT_DIR}
!python main.py infer /content/tygz-001 /content/tygz-001_output -s s -x  -p 1..3

/content/ndlocr_cli
start inference !
input_root : /content/tygz-001
output_root : /content/tygz-001_output
config_file : config.yml
[WARNING] save_xml and save_image flags are ignored because this is partial execution.
          All output of last proc will be saved in output directory.
load from config=src/ndl_layout/models/ndl_layout_config.py, checkpoint=src/ndl_layout/models/epoch_140_all_eql_bt.pth
set up EQL (version NDL), 9 classes included.
load checkpoint from local path: src/ndl_layout/models/epoch_140_all_eql_bt.pth
No Transformation module specified
No SequenceModeling module specified
model input parameters 32 1200 20 1 512 256 7085 100 None ResNet None CTC
loading pretrained model from src/text_recognition/models/ndlenfixed64-mj0-synth1.pth
[{'input_dir': '/content/tygz-001', 'img_list': ['/content/tygz-001/img/tygz-0010001-01.jpg', '/content/tygz-001/img/tygz-0010001-02.jpg', '/content/tygz-001/img/tygz-0010001-03.jpg', '/content/tygz-001/img/tygz-0010001-04.jpg', '/con

# 10. 結果の確認

In [ ]:
import glob
import os
for fpath in sorted(glob.glob("/content/tygz-001_output/tygz-001/txt/*_main.txt")):
    with open(fpath) as f:
        txtdata=f.read()
        print(os.path.basename(fpath).replace("_main.txt",""))
        print(txtdata)